In [ ]:
# PAPER 8 AND 9. User-user similarity matrix calculation 
#Import Python Libraries
import numpy as np
import pandas as pd
from IPython.display import display, HTML



In [ ]:
# PREPROCESSING

df=pd.read_csv('MovieLens.csv')
display(HTML(df.head(2).to_html()))

# vector for each user
df_ratings = df.pivot(index='userId', columns='movieId', values='rating')
print('DF of the ratings of each user - vector users')

# Fill missing values with 0
df_ratings = df_ratings.fillna(0)

sdf = df_ratings.astype(pd.SparseDtype("float", 0.0))
display(HTML(sdf.head(2).to_html()))

print('dense : {:0.2f} bytes'.format(df_ratings.memory_usage().sum() / 1e3))
print('sparse: {:0.2f} bytes'.format(sdf.memory_usage().sum() / 1e3))



In [ ]:

print('STEP 1. PAPER 8 AND 9.')
print('REPRESENTATION OF EACH USER BY THE DISTRIBUTION OF USER-USER SIMILARITY')
# PAPER 8 AND 9. STEP 1.
# USER-USER SIMILARITY MATRIX


# method to calculate the Constrained Pearson Correlation (CPC)
# @param u - user u's ratings
# @param v- v's ratings
# @param median - median rating in a rating range 

# @return Constrained PCC Correlation (CPC)

import math
def cpc(user1, user2, median):
    sumNum = 0
    sumDen1 = 0
    sumDen2 = 0
    for i in range(0, len(user1)):
        ui = user1[i] - median;
        vi = user2[i] - median; 
        
        sumNum += ui * vi
        sumDen1 += math.pow(ui,2)
        sumDen2 += math.pow(vi,2)    
    return sumNum / (math.sqrt(sumDen1)*math.sqrt(sumDen2))


# getting the userId index
array_index = df['userId'].unique()

# sim_matrix2: matrix of user-user similarities
sim_matrix2 = pd.DataFrame({'userId': array_index})
sim_matrix2.index = sim_matrix2.userId

med = df['rating'].median()

# aux variables: 
#      tuple_ids: tuples of ids
#      visited: set of tuples {(userA,userB), (userA, userC),...}, ids already visited

visited = set()
tuple_ids = tuple()
# We go through all the users
for i in sdf.index:
    cpc_array = np.empty(0, dtype=float)    
    
    # twice, to compare 2 users
    for j in sdf.index:         
        arr_ids =[sdf.loc[i].name, sdf.loc[j].name]
        arr_ids.sort()
        tuple_ids = tuple(arr_ids)
        if tuple_ids in visited:
            print('aaaa', tuple_ids)
            cpc_array = np.append(cpc_array, '-')
            continue
        cpcc = cpc(sdf.loc[i].tolist(), sdf.loc[j].tolist(), med)
        cpc_array = np.append(cpc_array, cpcc)
        visited.add(tuple_ids)

    sim_matrix2[sdf.index[i-1]] = cpc_array.tolist()
    print("similarity matrix:")   
    display(HTML(sim_matrix2.head(5).to_html()))
    
sim_matrix2.to_csv(r'sim_matrix_paper89-full.csv')


In [ ]:
# PRE-PROCESSING PART. - JUST ONCE, TO GET THE CSV OF THE FULL MATRIX

# 1. It is required to add the redundant values to the matrix

sim_matrix2 = sim_matrix.copy(deep=True)

for i in range(len(sim_matrix2)):
    for j in range(0, i):
        sim_matrix2.iloc[[j],[i]] = sim_matrix2.iloc[i][j]
    display(HTML(sim_matrix2.head(7).to_html()))
    
sim_matrix2.to_csv('sim_matrix_paper9-wholematrix.csv')